# Challenge 8: 自駕車接收命令與指揮 (計算理論 / 系統軟體)

2020.11.27

## Objective

- Learn language grammar to create own language which handled by specific application
- Understand regular language and context free language
- Learn useful command-line skill to leverage various system software
- Understand how application leverage system software

## Requirement

實作出小鴨車語 `DuckieDuckie2` 直譯器，並可利用環境變數調整速度與角速度

- 小鴨車語 `DuckieDuckie2`
    - 支援多句 `DuckieDuckie1` 命令 (表達式)
    - 新增迴圈
- 利用環境變數 environment variable 去調整 `前進` `後退` `右轉` `左轉` 時的速度與角速度
    - 方便在不修改程式碼的情況下，改變與設定程式的行為

> `DuckieDuckie2` 的語法及語義在 [Reference](#Reference)


## 小鴨車語 `DuckieDuckie2`

整個檔案可能是下列兩種格式

- 格式1. 無迴圈

具有多個 `DuckieDuckie1` 命令 (表達式)，依照順序執行  
每一句 `DuckieDuckie1` 命令還是遵循原本的格式與意義

- 格式2. 有迴圈

以 `loop` 開頭作為關鍵字，只會出現在檔案開頭，前面不會有其它字元  
`loop` 後面隔一個空白接著至少一位的數字，代表該迴圈需要被重複執行幾次  
數字後面隔著一個空白接著 `{`，在 `{` 後可以有任意個空白與換行 `\n`  
接著是多個 `DuckieDuckie1` 命令 (表達式)，這些是重複執行的部分  
在最後會有 `}` 將整個迴圈區間關閉  
在 `}` 後面可以有任意個空白或換行 `\n`


## Test Example

小鴨車語 `DuckieDuckie2` 的使用範例與行為

有一個檔案 `main.duckie`
```
forward 1500ms;
right 1s;
```

透過命令列 `python duckieduckie2.py < main.duckie` 使用輸入重定向

預期會看到小鴨車的行為是：

```
前進 1500 毫秒，右轉 1 秒，最後停止
```

更多範例請參考下面


### Example 1: 前進 1 秒，右轉 1 秒，重複 4 次

檔案 `example1.duckie`
```
loop 4 {
    forward 1s;
    right 1s;
}
```
連入小鴨車，並使用  
`python duckieduckie2.py < example1.duckie`  
會看到小鴨車行為是：
```
前進 1 秒，右轉 1 秒，前進 1 秒，右轉 1 秒，前進 1 秒，右轉 1 秒，前進 1 秒，右轉 1 秒，最後停止
```


### Example 2: 迴圈語法錯誤

檔案 `example2.duckie`
```
loop 2 
    backward 500ms;
    right 1s;
}

```
連入小鴨車，並使用  
`python duckieduckie2.py < example2.duckie`  
會看到小鴨車行為是：
```
停止
```

因為**語法是錯的，應該提示使用者語法有誤**，提示有錯誤即可，不需給出是什麼錯誤  
錯誤的地方在於代表迴圈次數的數字要隔一個空白接著 `{`


### Example 3: 迴圈語法錯誤

檔案 `example3.duckie`
```
loop 2 {
    loop 4 {
        forward 1s;
        left 1s;
    }
    right 2s;
}

```
連入小鴨車，並使用  
`python duckieduckie2.py < example3.duckie`  
會看到小鴨車行為是：
```
停止
```

因為**語法是錯的，應該提示使用者語法有誤**，提示有錯誤即可，不需給出是什麼錯誤  
錯誤的地方在於迴圈的 `{` 與 `}` 內只能是多個 `DuckieDuckie1` 命令 (表達式)


### Example 4: 語法錯誤 

檔案 `example4.duckie`
```
loop 4 {
    forward 1s;
    left 1s;
}
right 2s;
```
連入小鴨車，並使用  
`python duckieduckie2.py < example4.duckie`  
會看到小鴨車行為是：
```
停止
```

因為**語法是錯的，應該提示使用者語法有誤**，提示有錯誤即可，不需給出是什麼錯誤  
錯誤的地方在於 `}` 後只能是任意個空白或換行 `\n`，而不是另一個 `DuckieDuckie1` 命令


### Example 5: 語法錯誤 

檔案 `example5.duckie`
```
forward 1s;
left 2s
```
連入小鴨車，並使用  
`python duckieduckie2.py < example5.duckie`  
會看到小鴨車行為是：
```
停止
```

因為**語法是錯的，應該提示使用者語法有誤**，提示有錯誤即可，不需給出是什麼錯誤  
第二個 `DuckieDuckie1` 缺了分號 `;`，如果已經執行了 `forward 1s;` 是可以接受的


## 環境變數 environment variable

跑在作業系統內的程式，可以取得的外部環境變數，型態為字串 `str`

例如：可以有一個變數叫做 `DUCKIE_VELOCITY`，可以設定前進後退的速度

In [1]:
%%writefile show_env.py
import os
print(os.getenv("DUCKIE_VELOCITY"))

Overwriting show_env.py


接下來可以嘗試執行 `show_env.py` 其內容是上面的範例程式

In [2]:
! python show_env.py
! DUCKIE_VELOCITY=0.3 python show_env.py

None
0.3


### 環境變數的賦值

如果想在 `windows` 去設定環境變數

是 `powershell`請用
```
$env:DUCKIE_VELOCITY=0.3
python show_env.py
```

是 `cmd` 請用
```
set DUCKIE_VELOCITY=0.3
python show_env.py
```

> 如果想合併成一行執行，在中間加上 `&`

如果想在 `mac` 或 `linux` 等 unix-like 去設定環境變數

```
$ DUCKIE_VELOCITY=0.3 python show_env.py
```

或想要一次設定多個環境變數

```
$ DUCKIE_VELOCITY=0.3 DUCKIE_OMEGA=0.2 python show_env.py
```

或是

```
$ export DUCKIE_VELOCITY=0.3
$ python show_env.py
```



### 環境變數使用的範例

In [ ]:
import os

from lib.car_control import CarControl
    
car = CarControl(0.0)
    
cmd = "forward"

# no one set this environment variable, use default value
if os.getenv("DUCKIE_VELOCITY") is None:
    velocity = 0.3
else:
    # get setting value from environment variable
    velocity = float(os.getenv("DUCKIE_VELOCITY"))

# no one set this environment variable, use default value
if os.getenv("DUCKIE_OMEGA") is None:
    omega = 4
else:
    # get setting value from environment variable
    omega = float(os.getenv("DUCKIE_OMEGA"))

if cmd == "forward":
    car.move(velocity=velocity, omega=0)
elif cmd == "backward":
    car.move(velocity=-velocity, omega=0)
elif cmd == "left":
    car.move(velocity=0, omega=omega)
elif cmd == "right":
    car.move(velocity=0, omega=-omega)

## Demo

用小鴨車走出特定圖形，不用到非常精準，也沒有需要一次成功  
但會需要去利用 環境變數 去微調速度與角速度

同時，還會測試幾個語法會錯誤的例子


## Notice

- 因需要使用到 `duckiebot_cs_zoo/libs` 目錄下的函式庫，請將程式碼放置於 `duckiebot_cs_zoo` 目錄下
- 有任何問題，歡迎發問

## Reference

- [Environment variable wiki](https://en.wikipedia.org/wiki/Environment_variable)

## 小鴨車語 `DuckieDuckie2`

### 語法 Grammar

小鴨車語 `DuckieDuckie2` 的上下文語法定義為

$$
\begin{align}
G = \{ \sum, N, S, P \} \\
G \space \text{is grammar} \\
\sum \space \text{is alphabet and terminal symbol} \\
N \space \text{is nonterminal symbol} \\
S \space \text{is start symbol} \\
P \space \text{is production rules}
\end{align}
$$

- `alphabet` 字母集為 `a-z`, `0-9`, `;`, 換行 `\n` 與 空白 ` ` ，亦指 `terminal symbol` 終端符號
- `nonterminal symbol` 非終端符號有 `CMD`, `DIGIT`, `SCALAR`, `UNIT`, `TRIM`, `LOOP`, `DUCKIEDUCKIE1`, `DUCKIEDUCKIE2`
- `start symbol` 開始符號為 `DUCKIEDUCKIE2`
- `production rules` 產生規則較為複雜，其中使用到幾個符號
  - 使用三個正則運算：
    - 用 $\cup$ 表示 Union 聯集
    - 用 $\cdot$ 表示 Concatenate 連接
    - 用 $*$ 表示 Kleene Star 星號  
  - 使用單引號 $\text{'}$ 表達字母集中的終端符號  
  - 沒有單引號也非正則運算表達非終端符號

$$
\begin{align}
\text{CMD} &\to \text{'forward'} \cup \text{'backward'} \cup \text{'left'} \cup \text{'right'} & \\
\text{DIGIT} &\to \text{'0'} \cup \text{'1'} \
\cup \text{'2'} \cup \text{'3'} \cup \text{'4'} \cup \text{'5'} \
\cup \text{'6'} \cup \text{'7'} \cup \text{'8'} \cup \text{'9'} & \\
\text{SCALAR} &\to \text{DIGIT} \cdot \text{DIGIT}^* & \\
\text{UNIT} &\to \text{'ms'} \cup \text{'s'} & \\
\text{TRIM} &\to \text{' '} \cup \text{'\\n'} & \\
\text{LOOP} &\to \text{'loop'} & \\
\\
\text{DUCKIEDUCKIE1} &\to \text{CMD} \cdot \text{' '} \cdot \text{' '}^* \cdot \text{SCALAR} \cdot \text{UNIT} \cdot \text{';'} \cdot \text{TRIM}^* & \\
\text{DUCKIEDUCKIE2} &\to \text{DUCKIEDUCKIE1}^* \cup \text{LOOP} \cdot \text{' '} \cdot \text{SCALAR} \cdot \text{' '} \cdot \text{'\{'} \cdot \text{TRIM}^* \cdot \text{DUCKIEDUCKIE1}^* \cdot \text{'\}'} \cdot \text{TRIM}^*
\end{align}
$$


### 語義 Semantics

`DuckieDuckie1` 的非終端符號會被分成四個字串，分別是 `CMD`、`SCALAR`、`UNIT` 與 `TRIM`

- `CMD` 對應到小鴨車應該要有的行為
  - 目前 `DuckieDuckie1` 有四種行為
    - `forward` 前進
    - `backward` 後退
    - `left` 左轉
    - `right` 右轉
- `SCALAR` 對應到 `CMD` 行為持續時間的多寡，與 `UNIT` 配合表示不同量級的參數
- `UNIT` 對應到 `SCALAR` 的單位
- `TRIM` 在語義上沒有意義，應該忽略

其對應語義為：控制小鴨車進行 `CMD` 行為，持續時間為 `SCALAR`，時間單位為 `UNIT`

舉例而言，當 `CMD='forward', SCALAR='1', UNIT='s'`  
對應語義即為，小鴨車前進一秒

`DuckieDuckie2` 有兩種情況

- 情況1: 多個 `DuckieDuckie1`
  - 按照順序，等前一個 `DuckieDuckie1` 的語義結束後，處理下一個

舉例而言，當遇到 `CMD='forward', SCALAR='1', UNIT='s'` 接著 `CMD='backward', SCALAR='1', UNIT='s'`
對應語義即為，小鴨車前進一秒，接著後退一秒

- 情況2: 遇到 `LOOP` 迴圈
  - 後面接著的 `SCALAR` 代表該迴圈要重複幾次
  - 重複的部分為被 `{` 與 `}` 包圍起來的部分

舉例而言，當遇到 `SCALAR=2`，且 `{` `}` 包圍的部分是  `CMD='forward', SCALAR='1', UNIT='s'` 接著 `CMD='right', SCALAR='1', UNIT='s'`  
對應語義即為，小鴨車前進一秒，接著右轉一秒，再前進一秒，最後右轉一秒
